In [1]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings


In [2]:
loader = TextLoader('Sample.txt')
documents = loader.load()

In [3]:
len(documents)

1

In [5]:
text_splitter = CharacterTextSplitter(chunk_size=200,
                                      chunk_overlap=0)

text = text_splitter.split_documents(documents)
len(text)

Created a chunk of size 486, which is longer than the specified 200
Created a chunk of size 426, which is longer than the specified 200
Created a chunk of size 419, which is longer than the specified 200
Created a chunk of size 498, which is longer than the specified 200
Created a chunk of size 319, which is longer than the specified 200
Created a chunk of size 436, which is longer than the specified 200
Created a chunk of size 309, which is longer than the specified 200
Created a chunk of size 365, which is longer than the specified 200
Created a chunk of size 288, which is longer than the specified 200
Created a chunk of size 320, which is longer than the specified 200
Created a chunk of size 354, which is longer than the specified 200
Created a chunk of size 263, which is longer than the specified 200
Created a chunk of size 339, which is longer than the specified 200
Created a chunk of size 338, which is longer than the specified 200
Created a chunk of size 366, which is longer tha

19

In [6]:
embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

C:\Users\tinus\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
.gitattributes: 100%|██████████| 1.23k/1.23k [00:00<00:00, 1.23MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 191kB/s]
README.md: 100%|██████████| 10.7k/10.7k [00:00<00:00, 11.6MB/s]
config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 106kB/s]
data_config.json: 100%|██████████| 39.3k/39.3k [00:00<00:00, 476kB/s]
model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]Error while downloading from https://cdn-lfs.huggingface.co/sentence-transformers/all-MiniLM-L6-v2/53aa51172d142c89d9012cce15ae4d6cc0ca6895895114379cacb4fab128d9db?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1717495159&Policy

In [7]:
db = Chroma.from_documents(text, embeddings)

C:\Users\tinus\AppData\Roaming\Python\Python312\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [9]:
db._collection.get(include=['embeddings'])

{'ids': ['03c52237-da65-4a04-9746-5bc91e7624f1',
  '0f9224d0-485c-46fc-8cd3-92f7ba9a2f75',
  '12b0ec3a-ea1c-458e-be1d-d3f1279fe546',
  '15904d30-5cfe-4751-8131-3024fab1662f',
  '1a6d4cbc-409a-4b67-985d-f666d527dc43',
  '3311e4d0-86ca-4278-b1ca-09e9b1947c0a',
  '5e7b3afe-ac4d-40cb-8d2e-243a72917d32',
  '848587c1-aba7-45f3-a9f3-12398a6171f3',
  '8a1c3621-8ffe-42d2-a7a1-cf646b87a175',
  '8e1192a8-c004-40f7-9783-b057ac562383',
  '8e9788f8-f973-496f-a9aa-9e9e9bca2d32',
  '955582c5-bfb6-41fc-bf5a-15eae889b703',
  'a0599d93-2aa7-4604-8de1-26f0406f681d',
  'ac587fc7-b973-4f3c-905c-a2f6258fbd90',
  'af4f9d68-8122-47fa-b3e9-7505e6bfd465',
  'aff242b3-a581-4e98-bf37-e44017c0fc49',
  'bc73466c-6369-47b6-81eb-da6d7f3df133',
  'd79cf04e-4c25-40ab-b84f-95d6c7421403',
  'f58d583a-605a-4bfd-bde8-9cfd65e0b8aa'],
 'embeddings': [[0.03895367309451103,
   0.02619028650224209,
   -0.04022998735308647,
   0.03060499019920826,
   -0.02360576018691063,
   -0.0601368248462677,
   0.0739518329501152,
   0.016435

In [13]:
retriever = db.as_retriever(search_kwargs={"k":2})
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001D744ED54C0>, search_kwargs={'k': 2})

In [14]:
docs = retriever.get_relevant_documents("What is the capital of India")
docs

[Document(page_content='Delhi is the capital of India.', metadata={'source': 'Sample.txt'}),
 Document(page_content='India, officially known as the Republic of India, is a diverse and vibrant country located in South Asia. With a rich history spanning thousands of years, India is known for its cultural heritage, religious diversity, and vast landscapes. From the majestic Himalayas in the north to the serene backwaters of Kerala in the south, India encompasses a wide range of geographical features, including deserts, plains, mountains, and coastlines, making it a land of incredible natural beauty.', metadata={'source': 'Sample.txt'})]

In [1]:
import os
import openai
from langchain_community.llms.huggingface_hub import HuggingFaceHub
import pinecone
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_openai import OpenAI
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.chains.question_answering import load_qa_chain
from dotenv import load_dotenv
load_dotenv()

def load_docs(directory):
    loader = PyPDFDirectoryLoader(directory)
    documents = loader.load()
    return documents

directory = 'D:\\Data Science\\Generative_AI\\Data_connections'
documents = load_docs(directory)
#print(len(documents))

def split_docs(documents,chunk_size=1000, chunk_overlap=20):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    ) 
    docs = text_splitter.split_documents(documents)
    return docs

docs = split_docs(documents)
print(len(docs))

embeddings = SentenceTransformerEmbeddings(model_name= 'all-MiniLM-L6-V2')

query_result = embeddings.embed_query('Hello query')
print(len(query_result))

pc = pinecone.Pinecone(
    api_key=os.environ.get("pinecone_api_key")
)
index_name= "mcq-creator"
index = pc.Index(index_name)

index = Pinecone.from_documents(docs,embeddings, index_name=index_name)

def get_similar_docs(query, k):
    similar_docs = index.similarity_search(query, k=k)
    return similar_docs

llm = HuggingFaceHub(
    huggingfacehub_api_token= os.environ.get("huggingface_api"),
    repo_id="bigscience/bloom",
    model_kwargs={"temperature": 1e-10})

chain = load_qa_chain(llm, chain_type="stuff")

def get_answer(query):
    relavant_docs = get_similar_docs(query, k=1)
    #print(relavant_docs)
    response = chain.run(input_documents=relavant_docs,
                         question=query)
    return response

query_1 = "How is India's economy?"
answer = get_answer(query_1)
print(answer)


C:\Users\tinus\AppData\Roaming\Python\Python312\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


7
384


C:\Users\tinus\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
C:\Users\tinus\AppData\Roaming\Python\Python312\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

However, India also faces various socio-economic challenges. Poverty, income inequality, and 
unemployment are persistent issues that the country strives to address. Efforts are being made
to improve education, healthcare, infrastructure, and social welfare programs to uplift 
marginalized sections of society.
Education plays a vital role in India, with a strong emphasis on academic excellence. The 
country has a vast network of schools, colleges, and universities, producing a large number of 
graduates every year. Indian professionals have made significant contributions in various fields 
globally, particularly in science, technology, engineering, and mathematics (STEM).
The Indian film industry, popularly known as Bollywood, is a global phenomenon, producing the
largest number of films annually. Indian cinema reflects the 

In [8]:
import re
import json
from dotenv import load_dotenv
from langchain_openai.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
load_dotenv()

response_schemas = [
    ResponseSchema(name="question", description="Question generated from provided input text data."),
    ResponseSchema(name="choices", description="Available options for a multiple-choice question in comma seperated."),
    ResponseSchema(name="answer", description="Correct answer for the asked question.")
]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
print(output_parser)

format_instructions = output_parser.get_format_instructions()
print(format_instructions)

chat_model = ChatOpenAI()
print(chat_model)

prompt = ChatPromptTemplate(
    messages=[
        HumanMessagePromptTemplate.from_template(
            """When a text input is given by the user, please generate multiple choice questions
            from it along with the correct answer,
            \n{format_instructions}\n{user_prompt}
            """)
    ],
    input_variables=["user_prompt"],
    partial_variables={"format_instructions": format_instructions}
)

final_query = prompt.format_prompt(user_prompt = answer)
final_query.to_messages()
final_query_output = chat_model(final_query.to_messages())
print(final_query_output.content)

markdown_text = final_query_output.content
json_string = re.search(r'{(.*?)}', markdown_text, re.DOTALL).group(1)
print(json_string)

response_schemas=[ResponseSchema(name='question', description='Question generated from provided input text data.', type='string'), ResponseSchema(name='choices', description='Available options for a multiple-choice question in comma seperated.', type='string'), ResponseSchema(name='answer', description='Correct answer for the asked question.', type='string')]
The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"question": string  // Question generated from provided input text data.
	"choices": string  // Available options for a multiple-choice question in comma seperated.
	"answer": string  // Correct answer for the asked question.
}
```
client=<openai.resources.chat.completions.Completions object at 0x0000016298700F50> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000016298702180> openai_api_key=SecretStr('**********') openai_proxy=''
```json
{
	"question": "